In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime as dt
import hvplot.xarray

import glob
import os

from dask.distributed import LocalCluster
from skimage.measure import label

import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import PreProcess_Utils as pputil

sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/Modelling/Utilities')
import Snow_Utils as snow_util

In [2]:
cluster = LocalCluster(n_workers = 10, threads_per_worker = 10) # Refined through testing
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 127.65 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53200,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 127.65 GiB
Comm: tcp://127.0.0.1:53258,Total threads: 10
Dashboard: http://127.0.0.1:53264/status,Memory: 12.77 GiB
Nanny: tcp://127.0.0.1:53203,


2024-09-23 16:24:10,205 - distributed.nanny - ERROR - 
Traceback (most recent call last):
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\utils.py", line 832, in wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\nanny.py", line 552, in _on_worker_exit
    await self._unregister()
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\nanny.py", line 305, in _unregister
    await wait_for(
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\utils.py", line 1956, in wait_for
    return await fut
           ^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls\Lib\site-packages\distributed\core.py", line 1395, in send_recv_from_rpc
    return await send_recv(comm=comm, op=key, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\sno

In [ ]:
# %%time
# folder = r'C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\HBL\TemporalSnow\Fmask_V1\Products\interannual_tile'

# snowDynamics = xr.open_mfdataset(folder + '/*.nc', 
#                                  engine = 'netcdf4',
#                                  chunks = None).compute() # Added compute here, not sure what faster
# snowDynamics # ValueError: cannot reindex or align along dimension 'x' because the (pandas) index has duplicate values

In [ ]:
# variables_drop = variables[1:]
# variables_drop

In [ ]:
# %%time
# path = r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\HBL\TemporalSnow\Fmask_V1\Products\interannual_tile\HLS_Fmask_snowDynamics1823_828.nc"

# # test = pputil.uploadNC(path, type = 'dataset')
# test = xr.open_dataarray(path, engine = 'netcdf4', decode_coords = 'all', mask_and_scale = False, drop_variables = variables_drop)
# test # Works, but clunky

In [ ]:
# %%time
# path = r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\HBL\TemporalSnow\Fmask_V1\Products\interannual_tile\HLS_Fmask_snowDynamics1823_990.nc"
# variable = 'snow_end_q'

# test = pputil.uploadNC(path, type = 'dataset', chunks = {'x': -1, 'y': -1})[variable]
# test

In [ ]:
# test.rio.to_raster('snow_end_q_990.tif')

In [ ]:
# Load full files as chunked datasets (for loop through paths)
# Grab single variable as dataarray from each and combine_by_coords

In [7]:
%%time
folder = r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\HBL\TemporalSnow\Fmask_V1\Products\interannual_tile"
#folder = 'C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/HBL/TemporalSnow/Fmask_V1/Products2/interannual_tile' 
os.chdir(folder)

ncs = []
#variable = 'snow_start_mn'

for nc in os.listdir(folder):
    if nc.endswith('.nc'):
        path = os.path.abspath(nc) # Get full path of NetCDF file
        cube = pputil.uploadNC(path, type = 'dataset', chunks = {'x': -1, 'y': -1}) #[variable]
        ncs.append(cube)

CPU times: total: 188 ms
Wall time: 672 ms


In [8]:
ncs[0]

<xarray.Dataset> Size: 240MB
Dimensions:                       (x: 2001, y: 2001)
Coordinates:
  * x                             (x) float64 16kB 6.49e+05 ... 7.09e+05
  * y                             (y) float64 16kB 5.953e+05 ... 5.353e+05
    spatial_ref                   int64 8B ...
    band                          <U5 20B ...
Data variables: (12/15)
    snow_start_mn                 (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    snow_end_mn                   (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    snow_length_mn                (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    snow_periods_mn               (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    pPerennialSnow                (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    pSnowFree                     (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    ...                            ...
    snow_end_q                    (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    snow_length_u_mn              (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    snow_length_q                 (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    snowSeason_clearObsPerDay_mn  (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    snowFall_clearObsPerDay_mn    (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>
    snowMelt_clearObsPerDay_mn    (y, x) float32 16MB dask.array<chunksize=(2001, 2001), meta=np.ndarray>

In [ ]:
#ncs

In [ ]:
# ncs[0]

In [ ]:
# %%time

# das = []
# variable = 'snow_end_mn'

# for da in range(len(ncs)):
#     das.append(ncs[da][variable]) 

In [ ]:
# das

In [ ]:
# %%time
# #var_roi = xr.combine_by_coords(das) # There is one pixel overlap between tiles, combine_by_coords does not play nice with this
# #var_roi = xr.merge(das, compat = 'no_conflicts') # Not what I want either...
# var_roi = xr.concat(das, 'newdim', join = 'outer').mean('newdim', skipna = True) # Works, but may nto scale well to many tiles... 

# # https://stackoverflow.com/questions/62806175/xarray-combine-by-coords-return-the-monotonic-global-index-error 
# #(trim edges of all and combine_by_coords?)

# var_roi

In [4]:
%%time
# What about running this on full dataset rather than variable by variable?
roi = xr.concat(ncs, 'newdim', join = 'outer').mean('newdim', skipna = True)
roi

CPU times: total: 812 ms
Wall time: 5.66 s


<xarray.Dataset> Size: 2GB
Dimensions:                       (y: 6001, x: 6001)
Coordinates:
  * x                             (x) float64 48kB 6.49e+05 ... 8.29e+05
  * y                             (y) float64 48kB 5.353e+05 ... 7.153e+05
    spatial_ref                   int64 8B 0
    band                          <U5 20B 'Fmask'
Data variables: (12/15)
    snow_start_mn                 (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>
    snow_end_mn                   (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>
    snow_length_mn                (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>
    snow_periods_mn               (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>
    pPerennialSnow                (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>
    pSnowFree                     (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>
    ...                            ...
    snow_end_q                    (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>
    snow_length_u_mn              (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>
    snow_length_q                 (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>
    snowSeason_clearObsPerDay_mn  (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>
    snowFall_clearObsPerDay_mn    (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>
    snowMelt_clearObsPerDay_mn    (y, x) float32 144MB dask.array<chunksize=(6001, 6001), meta=np.ndarray>

In [ ]:
#roi.nbytes / 1e9

In [ ]:
%%time
path = r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\HBL\TemporalSnow\Fmask_V1\Products\interannual_roi"
variables = ['snow_start_mn', 'snow_end_mn', 'snow_length_mn', 'snow_periods_mn', 'pPerennialSnow', 'pSnowFree', 'snow_start_u_mn', 
             'snow_start_q', 'snow_end_u_mn', 'snow_end_q', 'snow_length_u_mn', 'snow_length_q', 'snowSeason_clearObsPerDay_mn', 
             'snowFall_clearObsPerDay_mn', 'snowMelt_clearObsPerDay_mn']

for variable in variables:
    da = roi[variable] # Single snow dynamic stat from dataset
    da.rio.write_nodata(-999, encoded = True, inplace = True) # Ensured NaN NoData values are saved
    da.rio.to_raster(os.path.join(path, 'HLS_Fmask_' + variable + '1823_roi.tif'))

In [ ]:
# roi['snow_end_mn'].plot()

In [ ]:
# np.unique(roi['snow_end_mn'], return_counts = True)

In [ ]:
# roi['snow_end_mn'].rio.to_raster('test1.tif')

In [ ]:
#roi['snow_end_mn'].isnull().sum().compute()

In [ ]:
#ncs[0]['snow_end_mn'].rio.nodata

In [ ]:
# test = roi['snow_end_mn']
# test.rio.write_nodata(-999, encoded = True, inplace = True)
# test.rio.nodata

In [ ]:
# test.rio.to_raster('test4.tif')